In [1]:
# %load ../loaders/imports.py
%load_ext autoreload
%autoreload 2
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import time
import pdb

sys.path.append('..')

from utils import gen_covariance, gen_beta2, gen_data, get_cov_list
from utils import selection_accuracy
from sklearn.linear_model import LassoLars, lasso_path, LinearRegression

from pyuoi.linear_model import UoI_Lasso
from pyuoi.linear_model.adaptive import (mBIC, eBIC, mBIC2, BIC, MIC, 
bayesian_log_ll, bayesian_lambda_selection, L1_bayes)
from sklearn.linear_model import lars_path
from sklearn.preprocessing import StandardScaler
from scipy.special import binom

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
# Sparsity estimation on the basis of BIC
def sparsity_estimator0(X, y, n_boots = 48, train_frac = 0.75):
    
    sparsity_estimates = np.zeros(n_boots)
    
    n_features, n_samples = X.shape
    
    for boot in range(n_boots):
        # Draw bootstraps
        idxs_train, idxs_test = train_test_split(np.arange(X.shape[0]), 
                                                 train_size = train_frac,
                                                 test_size = 1 - train_frac)
        Xb = X[idxs_train]
        yb = y[idxs_train]

        Xb = StandardScaler().fit_transform(Xb)
        yb -= np.mean(yb)

        alphas, _, coefs  = lars_path(Xb, yb.ravel(), method = 'lasso')

        y_pred = Xb @ coefs

        # Assess BIC on the LARS path to estimate the sparsity. 
        BIC_scores = np.array([MIC(yb.ravel(), y_pred[:, j].ravel(),
                                   np.count_nonzero(coefs[:, j]), np.log(n_samples)) 
                               for j in range(coefs.shape[1])])  
        
        sparsity_estimates[boot] = float(np.count_nonzero(
                            coefs[:, np.argmin(BIC_scores)]))/float(n_features)

    return sparsity_estimates
        

In [4]:
# Refine sparsity estimate using mBIC
def sparsity_estimator1(X, y, s0, n_boots = 48, train_frac = 0.75):
        
    sparsity_estimates = np.zeros(n_boots)
    
    for boot in range(n_boots):
        # Draw bootstraps
        idxs_train, idxs_test = train_test_split(np.arange(X.shape[0]), 
                                                 train_size = train_frac,
                                                 test_size = 1 - train_frac)
        Xb = X[idxs_train]
        yb = y[idxs_train]

        Xb = StandardScaler().fit_transform(Xb)
        yb -= np.mean(yb)

        alphas, _, coefs  = lars_path(Xb, yb.ravel(), method = 'lasso')

        y_pred = Xb @ coefs
        
        mBIC_scores = np.zeros(coefs.shape[1])
        
        for j in range(coefs.shape[1]): 
        
            ll_, p1_, BIC_, BIC2_, BIC3_, M_k_, P_M_ = bayesian_lambda_selection(yb, y_pred[:, j], n_features, 
                                                                                 np.count_nonzero(coefs[:, j]),
                                                                                 s0, 0)
            mBIC_scores[j] = 2 * ll_ - BIC_ - BIC2_ + BIC3_ + P_M_ 
            
        sparsity_estimates[boot] = float(np.count_nonzero(
                            coefs[:, np.argmax(mBIC_scores)]))/float(n_features)

    return sparsity_estimates

In [5]:
# Fit a model using adaptive BIC criteria given sparsity estimates
def adaptive_BIC_estimator(X, y, sparsity_estimates, n_boots = 48, train_frac = 0.75):
        
    n_samples, n_features = X.shape
    
    np1 = 100
    penalties = np.linspace(0, 5 * np.log(n_samples), np1)
    
    oracle_penalty = np.zeros(n_boots)
    bayesian_penalty = np.zeros(n_boots)

    estimates = np.zeros((n_boots, n_features))
    oracle_estimates = np.zeros((n_boots, n_features))

    
    for boot in range(n_boots):
        # Draw bootstraps
        idxs_train, idxs_test = train_test_split(np.arange(X.shape[0]), 
                                                 train_size = train_frac,
                                                 test_size = 1 - train_frac)
        Xb = X[idxs_train]
        yb = y[idxs_train]

        Xb = StandardScaler().fit_transform(Xb)
        yb -= np.mean(yb)

        alphas, _, coefs  = lars_path(Xb, yb.ravel(), method = 'lasso')

        supports = (coefs.T != 0).astype(bool)

        # Stick the true model in there
        # supports = np.vstack([supports, (beta.ravel() !=0).astype(bool)])

        sa = selection_accuracy(beta.ravel(), supports)
        # Keep track of oracle performance
        MIC_scores_ = np.zeros((supports.shape[0], np1))

        boot_estimates = np.zeros((supports.shape[0], n_features))

        models = []
        
        for j in range(supports.shape[0]):
            
            support = supports[j, :]

            if np.count_nonzero(1 * support > 0):
                model = LinearRegression().fit(X[:, support] , y)
                boot_estimates[j, support] = model.coef_.ravel()
                y_pred = model.predict(X[:, support])
                models.append(model)
            else:
                y_pred = np.zeros(y.size)
                models.append(np.nan)
                
            MIC_scores_[j, :] =  np.array([MIC(y.ravel(), y_pred.ravel(), 
                                           np.count_nonzero(1 * support), penalty) 
                                           for penalty in penalties])

        selected_models = np.argmin(MIC_scores_, axis = 0)
        
        bayes_factors = np.zeros(np1)

        for i3, penalty in enumerate(penalties):

            support = supports[selected_models[i3], :]
            yy = models[selected_models[i3]].predict(X[:, support])

            ll_, p1_, BIC_, BIC2_, BIC3_, M_k_, P_M_ = bayesian_lambda_selection(
                                                       y, yy, n_features, 
                                                       np.count_nonzero(1 * support),
                                                       sparsity_estimates, penalty)

            # Add things up appropriately
            bayes_factors[i3] = 2 * ll_ - BIC_ - BIC2_ + BIC3_ - p1_ + P_M_


        # Select the penalty based on the highest bayes factors 
        bidx = np.argmax(bayes_factors)

        # Save the penalty strength and the chosen model
        bayesian_penalty[boot] = penalties[bidx]

        # For MIC scores, record the oracle selection accuracy and the oracle penalty
        MIC_selection_accuracies = [selection_accuracy(beta.ravel(), 
                                    supports[selected_models[j], :]) 
                                    for j in range(selected_models.size)]

        # For MIC scores, record the oracle selection accuracy and the orcale penalty 

        oracle_penalty[boot] = penalties[np.argmax(MIC_selection_accuracies)]
#        MIC_oracle_sa_[boot] = np.max(MIC_selection_accuracies)    
#        bMIC_sa_[boot] = MIC_selection_accuracies[bidx]
        
        # Record the best estimate on this bootstrap as determined by the oracle
        # and by our procedure
        estimates[boot, :] = boot_estimates[selected_models[bidx], :]
        oracle_estimates[boot, :] = \
        boot_estimates[selected_models[np.argmax(MIC_selection_accuracies)], :]

    # Take the median and record final selection accuracies
    final_bayes_estimates = np.median(estimates, axis = 0)
    final_oracle_estimates = np.median(oracle_estimates, axis = 0)

    return final_bayes_estimates, final_oracle_estimates, oracle_penalty, bayesian_penalty, estimates
    

In [7]:
n_features = 50
ns = 200
sparsity = np.linspace(0.05, 1, 11)

# Covariance of design matrix
sigma = gen_covariance(n_features, 0, n_features, 5, 1)

n_boots = 48
train_frac = 0.75

# Iteratively feed the sparsity estimate back into the algorithm
n_iter = 5

# Things to be recorded:
sparsity_estimates = np.zeros((sparsity.size, n_iter + 2,  n_boots))
# bayesian_penalty = np.zeros((sparsity.size, n_boots))
# oracle_penalty = np.zeros((sparsity.size, n_boots))
# MIC_oracle_sa_ = np.zeros((sparsity.size, n_boots))
# bMIC_sa_ = np.zeros((sparsity.size, n_boots))
# MIC_oracle_sa = np.zeros(sparsity.size)
# bMIC_sa = np.zeros(sparsity.size)

final_bayes_estimates = np.zeros((sparsity.size, n_features))
final_oracle_estimates = np.zeros((sparsity.size, n_features))

for i, s in enumerate(sparsity):
    t0 = time.time()
    beta = gen_beta2(n_features, n_features, sparsity = s, betawidth = 0)
    X, X_test, y, y_test, ss = gen_data(ns, n_features, kappa = 5, 
                                        covariance = sigma, beta = beta)

    # Initial sparsity estimates
    sparsity_estimates_ = sparsity_estimator0(X, y)
    sparsity_estimates[i, 0, :] = sparsity_estimates_
    # Refine 
    sparsity_estimates_ = sparsity_estimator1(X, y, sparsity_estimates_)
    sparsity_estimates[i, 1, :] = sparsity_estimates_

    
    for j in range(n_iter):
    
        # Fit
        fbe, foe, op, bp, estimates = adaptive_BIC_estimator(X, y, sparsity_estimates_)
        pdb.set_trace()

        # Obtain sparsity estimates 
        sparsity_estimates_ = np.count_nonzero(estimates, axis = 1).astype(float)/n_features
        sparsity_estimates[i, j + 2, :] = sparsity_estimates_
    
        
    
    print(time.time() - t0)


> <ipython-input-7-c33f474c835a>(47)<module>()
-> sparsity_estimates_ = np.count_nonzero(estimates, axis = 1).astype(float)/n_features
(Pdb) quit()


BdbQuit: 